In [1]:
# Get back to project root directory

import os
os.chdir("../")
os.getcwd()

'/home/jatin/Study/Live Courses/Projects/End-to-End-ML-Pipeline'

In [2]:
# Import necessary libraries

import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from ML_pipeline.constants import *
from ML_pipeline.utils.common import read_yaml, create_directories

In [3]:
# Define the Config entity

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio : float
    target_column: str

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH,
            schema_filepath= SCHEMA_FILE_PATH,
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path = config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column= schema.name
        )
        return model_trainer_config


In [9]:
import os
import pandas as pd
from ML_pipeline import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        el = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=1)
        el.fit(train_x,train_y)

        joblib.dump(el, os.path.join(self.config.root_dir,self.config.model_name))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.info(e)
    raise(e)

[2025-06-14 00:20:17,501: INFO: common: 2025-06-14 00:20:17,501]
[2025-06-14 00:20:17,503: INFO: common: 2025-06-14 00:20:17,503]
[2025-06-14 00:20:17,506: INFO: common: 2025-06-14 00:20:17,506]
[2025-06-14 00:20:17,507: INFO: common: 2025-06-14 00:20:17,507]
[2025-06-14 00:20:17,508: INFO: common: 2025-06-14 00:20:17,508]
